In [ ]:
from calendar import monthrange
from datetime import datetime
import itertools

from netCDF4 import date2num, num2date
import numpy as np
from pyproj import Proj
from siphon.catalog import TDSCatalog
import xarray as xr


In [14]:
#29.7604° N, 95.3698° W
base_catalog = 'http://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/NARR/pressure/catalog.xml'
main_cat = TDSCatalog(base_catalog)
points = [(35.25 , -97.1), (40, -105)]
lats, lons = map(np.array, zip(*points))
fields = ['air']



# Spatial location to find stuff
#points = [(28. , -94), (29., -95.)]
#lats, lons = map(np.array, zip(*points))
#fields = ['air', 'hgt', 'shum', 'uwnd', 'vwnd']


In [15]:
for year, month, var in itertools.product((2011, 2015, 2016, 2017), range(1, 13), fields):
    # Figure out what to grab
    dsname = '{}.{:4d}{:02d}.nc'.format(var, year, month)
    print('{}: downloading...'.format(dsname), end='')
    
    # Grab it using opendap--manually convert to CF to work around the
    # fact that missing_value and _FillValue differ
    ds = xr.open_dataset(main_cat.datasets[dsname].access_urls['OPENDAP'],
                         decode_cf=False)
    ds = xr.conventions.decode_cf(ds, mask_and_scale=False)

    # Grab the projection variable and convert our points to that
    # Probably not strictly necessary
    proj_var = ds[ds[var].grid_mapping]
    proj = Proj(proj='lcc', lat_0=proj_var.latitude_of_projection_origin,
            lon_0=proj_var.longitude_of_central_meridian,
            lat_1=proj_var.standard_parallel[0], lat_2=proj_var.standard_parallel[1],
            x_0=proj_var.false_easting, y_0=proj_var.false_northing, ellps='sphere')
    x, y = proj(lons, lats)


    # Subset the data
    print('subsetting...', end='')
    pt_ds = ds.sel_points(x=x, y=y, method='nearest')
    print('saving...', end='')

    # Save to disk
    pt_ds.to_netcdf(dsname)
    print('done.')

air.201101.nc: downloading...subsetting...

RuntimeError: NetCDF: Access failure

In [11]:
print(x)

[ 6989957.23359756  6867552.18319084]


In [ ]:
main_cat.datasets.keys()

In [ ]:
main_cat.datasets[dsname].access_urls['OPENDAP']

In [ ]:
import siphon
siphon.__version__

In [2]:
cat = TDSCatalog('https://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/NARR/pressure/catalog.xml')
ds = list(cat.datasets.values())[0]
ds.access_urls

{'HTTPServer': 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/NARR/pressure/air.197901.nc',
 'OPENDAP': 'https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/NARR/pressure/air.197901.nc',
 'WCS': 'https://www.esrl.noaa.gov/psd/thredds/wcs/Datasets/NARR/pressure/air.197901.nc',
 'WMS': 'https://www.esrl.noaa.gov/psd/thredds/wms/Datasets/NARR/pressure/air.197901.nc'}

In [ ]:
{'HTTPServer': 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/NARR/pressure/air.197901.nc',
 'OPENDAP': 'https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/NARR/pressure/air.197901.nc',
 'WCS': 'https://www.esrl.noaa.gov/psd/thredds/wcs/Datasets/NARR/pressure/air.197901.nc',
 'WMS': 'https://www.esrl.noaa.gov/psd/thredds/wms/Datasets/NARR/pressure/air.197901.nc'}